In [1]:

from gqimax.instructor import Instructor
%load_ext autoreload
%autoreload 2

ImportError: cannot import name 'instructor_to_lut' from 'gqimax.mapper' (c:\Users\Public\Documents\Github\qimax\gqimax\mapper.py)

In [8]:
ins = Instructor(4)
ins.append("h", 0)
ins.append("rx", 1, 0.78)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [0, 1])
ins.append("h", 2)
ins.append("h", 2)
ins.append("ry", 0, 0.56)
ins.append("cx", [1, 2])
ins.append("h", 1)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 2)
ins.append("cx", [1, 3])
ins.operatoring()

In [9]:
ins.create_lut_noncx(save_npy = False)


In [12]:
import numpy as np

non_zero_percentage = np.count_nonzero(ins.LUT) / ins.LUT.size * 100
print(f"Percentage of non-zero elements in ins.LUT: {non_zero_percentage:.2f}%")

Percentage of non-zero elements in ins.LUT: 29.17%


In [4]:
pstabilizer = PStabilizer(0, 3)
pstabilizer.map(ins)

In [5]:
pstabilizer.indices

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63], dtype=int64)

In [6]:
pstabilizer.lambdas

array([ 0.00894892, -0.06903402,  0.09085608, -0.01273684,  0.02770156,
       -0.01464247,  0.03290772,  0.04842204,  0.02993926, -0.01005337,
        0.02836496, -0.08535901,  0.00985582, -0.0169714 ,  0.02638212,
       -0.14737603,  0.01195674, -0.0022073 , -0.12578281,  0.04584833,
        0.07132364,  0.00384635, -0.0057283 ,  0.04653465,  0.03650245,
        0.01043427, -0.12022093, -0.02564443,  0.17502991,  0.12456996,
       -0.00324415,  0.0537573 ,  0.06101798, -0.12200946,  0.03589752,
       -0.02267155,  0.03263157,  0.0459531 , -0.15145757,  0.0654638 ,
        0.02262278,  0.16556499,  0.00933679, -0.014769  , -0.0302541 ,
       -0.03639622,  0.01148126, -0.0265554 ,  0.01506938,  0.02709955,
       -0.10852674,  0.01006342, -0.00274224, -0.03803595, -0.01703928,
        0.01342869, -0.09389169,  0.08872873,  0.1209212 ,  0.05687427,
       -0.08786091,  0.09199429,  0.11613111])